In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

In [102]:
X_train = pd.read_csv('svm_dataset/X_train.csv', header=None)
X_test = pd.read_csv('svm_dataset/X_test.csv', header=None)
y_train = pd.read_csv('svm_dataset/y_train.csv', header=None)
y_test = pd.read_csv('svm_dataset/y_test.csv', header=None)

X_train.head()

,0,1
0,-0.189839,1.050166
1,-0.132564,0.982741
2,-0.394263,1.221791
3,-0.268637,1.249947
4,1.948270,-0.478318


In [103]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [104]:
class FNN(nn.Module):
    def __init__(self, hidden_layers, hidden_layer_size, input_size=2, output_size=1):
        super(FNN, self).__init__()

        self.layers = []
        prev_layer_size = input_size

        for i in range(hidden_layers):
            self.layers.append(nn.Linear(prev_layer_size, hidden_layer_size))
            self.layers.append(nn.ReLU())
            prev_layer_size = hidden_layer_size

        # Output layer
        self.layers.append(nn.Linear(prev_layer_size, output_size))
        self.layers.append(nn.Sigmoid())

        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.model(x)

In [105]:
def train(model: nn.Module, train_loader):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    epochs = 20

    for epoch in range(epochs):
        model.train()

        for x_train, y_train in train_loader:
            optimizer.zero_grad()
            output = model(x_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')
    
    return model


def eval(model: nn.Module):
    model.eval()
    threshold = 0.5

    with torch.no_grad():
        y_train_pred = model(X_train_tensor)
        y_train_pred = (y_train_pred >= threshold)
        
        train_accuracy = accuracy_score(y_train.values, y_train_pred.numpy())
        print(f'\n\nTrain Accuracy: {train_accuracy}')

        y_test_pred = model(X_test_tensor)
        y_test_pred = (y_test_pred >= threshold)
        test_accuracy = accuracy_score(y_test.values, y_test_pred.numpy())
        print(f'Test Accuracy: {test_accuracy}')

(a) Train a network with 1 hidden layer, and the number of neurons for the hidden layer is 16.

In [106]:
model = FNN(1, 16)
train(model, train_loader)
eval(model)

Epoch 1/20, Loss: 0.6410
Epoch 2/20, Loss: 0.6208
Epoch 3/20, Loss: 0.6222
Epoch 4/20, Loss: 0.6012
Epoch 5/20, Loss: 0.5905
Epoch 6/20, Loss: 0.5740
Epoch 7/20, Loss: 0.5901
Epoch 8/20, Loss: 0.6127
Epoch 9/20, Loss: 0.5941
Epoch 10/20, Loss: 0.5300
Epoch 11/20, Loss: 0.5772
Epoch 12/20, Loss: 0.5236
Epoch 13/20, Loss: 0.5891
Epoch 14/20, Loss: 0.5056
Epoch 15/20, Loss: 0.5610
Epoch 16/20, Loss: 0.4940
Epoch 17/20, Loss: 0.5387
Epoch 18/20, Loss: 0.4794
Epoch 19/20, Loss: 0.5277
Epoch 20/20, Loss: 0.5192


Train Accuracy: 0.8375
Test Accuracy: 0.725


(b) Train a network with 2 hidden layers, and the number of neurons for each hidden layer is 16.

In [107]:
model = FNN(2, 16)
train(model, train_loader)
eval(model)

Epoch 1/20, Loss: 0.7136
Epoch 2/20, Loss: 0.6657
Epoch 3/20, Loss: 0.6653
Epoch 4/20, Loss: 0.6646
Epoch 5/20, Loss: 0.6706
Epoch 6/20, Loss: 0.6583
Epoch 7/20, Loss: 0.6541
Epoch 8/20, Loss: 0.6412
Epoch 9/20, Loss: 0.6564
Epoch 10/20, Loss: 0.6339
Epoch 11/20, Loss: 0.6221
Epoch 12/20, Loss: 0.6321
Epoch 13/20, Loss: 0.6214
Epoch 14/20, Loss: 0.5966
Epoch 15/20, Loss: 0.6188
Epoch 16/20, Loss: 0.5821
Epoch 17/20, Loss: 0.5612
Epoch 18/20, Loss: 0.5652
Epoch 19/20, Loss: 0.5539
Epoch 20/20, Loss: 0.5213


Train Accuracy: 0.8625
Test Accuracy: 0.75


(c) Train a network with 3 hidden layers, and the number of neurons for each hidden layer is 16.

In [108]:
model = FNN(3, 16)
train(model, train_loader)
eval(model)

Epoch 1/20, Loss: 0.6872
Epoch 2/20, Loss: 0.6856
Epoch 3/20, Loss: 0.6847
Epoch 4/20, Loss: 0.6915
Epoch 5/20, Loss: 0.6812
Epoch 6/20, Loss: 0.6771
Epoch 7/20, Loss: 0.6654
Epoch 8/20, Loss: 0.6638
Epoch 9/20, Loss: 0.6596
Epoch 10/20, Loss: 0.6429
Epoch 11/20, Loss: 0.6227
Epoch 12/20, Loss: 0.6272
Epoch 13/20, Loss: 0.5839
Epoch 14/20, Loss: 0.5697
Epoch 15/20, Loss: 0.5605
Epoch 16/20, Loss: 0.5302
Epoch 17/20, Loss: 0.5182
Epoch 18/20, Loss: 0.4902
Epoch 19/20, Loss: 0.4611
Epoch 20/20, Loss: 0.4782


Train Accuracy: 0.85
Test Accuracy: 0.8
